In [ ]:
from flask import Flask,request,jsonify,render_template



app = Flask(__name__) # ,static_url_path='/'

#@app.route('/hello')
#def index():
#    return render_template('index.html',name='John')  #{'hello': 'world'} 


@app.route('/get_p_acc_lvl', methods=['GET'])
def get_potential_accident_level():
    
    query_parameters = request.args
    
    acc_desc = query_parameters.get('description')
    
    prediction = 'Potential Accident Level XXX' #ft.predict('acc_desc')
    
    return jsonify(prediction)


if __name__ == '__main__':
  
    app.run()


In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.impute import SimpleImputer
# Basic packages
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns, gc
from scipy import stats; from scipy.stats import zscore, norm, randint
import matplotlib.style as style; style.use('fivethirtyeight')
# Models
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score, learning_curve
#from wordcloud import WordCloud, STOPWORDS

# Display settings
pd.options.display.max_rows = 400
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format

random_state = 42
np.random.seed(random_state)

# Suppress warnings
import warnings; warnings.filterwarnings('ignore')

In [9]:
df = pd.read_csv('industrial_safety_and_health_database_with_accidents_description.csv')

In [10]:
ds = df.copy()
ds.drop('Unnamed: 0',1, inplace = True)

In [14]:
# Defining the level as per Accident Level
def Accident_level_conversion(x):
  if x in ['I','II']:
    level = 'Low'
  elif x in ['III']:
    level = 'Medium'
  elif x in ['IV', 'V', 'VI']:
    level = 'High'
  return level

In [15]:
ds['Accident Severity'] = ds['Accident Level'].apply(Accident_level_conversion)

In [16]:
ds['Potential Severity'] = ds['Potential Accident Level'].apply(Accident_level_conversion)

In [18]:
replace_value = {'Local_01':1, 'Local_02':2, 'Local_03':3, 'Local_04':4, 'Local_05':5, 'Local_06':6, 'Local_07':7, 'Local_08':8, 
                 'Local_10':10, 'Local_09':9, 'Local_11':11, 'Local_12':12}
ds['Local'] = ds['Local'].map(replace_value)

replace_value = {'I':1, 'IV':4, 'III':3, 'II':2, 'V':5}
ds['Accident Level'] = ds['Accident Level'].map(replace_value)

replace_value = {'IV':4, 'III':3, 'I':1, 'II':2, 'V':5, 'VI':6}
ds['Potential Accident Level'] = ds['Potential Accident Level'].map(replace_value)

del replace_value

In [19]:
# Removong stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
ds['clean_Description'] = ds['Description'].apply(lambda x: ' '.join([words for words in x.split() if words not in stopwords]))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\reach\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
pip install fasttext

Note: you may need to restart the kernel to use updated packages.


In [21]:
import fasttext
import csv

In [23]:
ft_df_Potential = pd.DataFrame(columns=['fasttext_data'])
ft_df_Accident = pd.DataFrame(columns=['fasttext_data'])

ft_df_Potential['fasttext_data'] ='__label__' + ds['Potential Accident Level'].astype(str) + " "+ds['clean_Description']
ft_df_Accident['fasttext_data'] ='__label__' + ds['Accident Level'].astype(str) + " "+ds['clean_Description']

In [25]:
ft_df_Potential.to_csv(r'fasttext_industrial_Potential.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")
ft_df_Accident.to_csv(r'fasttext_industrial_Accident.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

In [26]:
!head -n 340 fasttext_industrial_Potential.txt > ft_ind_Potential.train
!tail -n 85 fasttext_industrial_Potential.txt > ft_ind_Potential.valid

In [27]:
!wc fasttext_industrial_Accident.txt

   425  16137 132303 fasttext_industrial_Accident.txt


In [28]:
!head -n 340 fasttext_industrial_Accident.txt > ft_ind_Accident.train
!tail -n 85 fasttext_industrial_Accident.txt > ft_ind_Accident.valid

In [29]:
ft_model_Potential = fasttext.train_supervised(input="ft_ind_Potential.train")

In [30]:
ft_model_Accident = fasttext.train_supervised(input="ft_ind_Accident.train")

In [31]:
ft_model_Potential.save_model("ft_model_Potential.bin")
ft_model_Accident.save_model("ft_model_Accident.bin")

In [32]:
ft_model_Potential.test("ft_ind_Potential.valid")
ft_model_Accident.test("ft_ind_Accident.valid")

(85, 1.0, 1.0)

In [34]:
!head -n 340 fasttext_industrial_Potential.txt > ft_ind_Potential.train
!tail -n 85 fasttext_industrial_Potential.txt > ft_ind_Potential.valid

In [35]:
!head -n 340 fasttext_industrial_Accident.txt > ft_ind_Accident.train
!tail -n 85 fasttext_industrial_Accident.txt > ft_ind_Accident.valid

In [36]:
ft_model_Potential = fasttext.train_supervised(input="ft_ind_Potential.train")
ft_model_Accident = fasttext.train_supervised(input="ft_ind_Accident.train")

In [38]:
ft_model_Potential.test("ft_ind_Potential.valid")

(85, 1.0, 1.0)

In [39]:
ft_model_Accident.test("ft_ind_Accident.valid")

(85, 1.0, 1.0)

In [40]:
ft_model_Potential = fasttext.train_supervised(input="ft_ind_Potential.train", lr=0.9)

In [41]:
ft_model_Accident = fasttext.train_supervised(input="ft_ind_Accident.train", lr=0.9)

In [42]:
ft_model_Potential.test("ft_ind_Potential.valid")

(85, 1.0, 1.0)

In [43]:
ft_model_Accident.test("ft_ind_Accident.valid")

(85, 1.0, 1.0)

In [44]:
#With both Lr and EPOC combined
ft_model_Potential = fasttext.train_supervised(input="ft_ind_Potential.train", lr=0.9, epoch=25)

In [45]:
ft_model_Accident = fasttext.train_supervised(input="ft_ind_Accident.train", lr=0.9, epoch=25)

In [46]:
ft_model_Potential.test("ft_ind_Potential.valid")

(85, 1.0, 1.0)

In [47]:
ft_model_Accident.test("ft_ind_Accident.valid")

(85, 1.0, 1.0)

In [48]:
#With WordNGrams
ft_model_Potential = fasttext.train_supervised(input="ft_ind_Potential.train", lr=0.8, epoch=25, wordNgrams=2)
ft_model_Accident = fasttext.train_supervised(input="ft_ind_Accident.train", lr=0.8, epoch=25, wordNgrams=2)

In [49]:
ft_model_Potential.test("ft_ind_Potential.valid")

(85, 1.0, 1.0)

In [50]:
ft_model_Accident.test("ft_ind_Accident.valid")

(85, 1.0, 1.0)

In [51]:
# With hierarchical softmax
ft_model_Potential = fasttext.train_supervised(input="ft_ind_Potential.train", lr=0.7, epoch=25, wordNgrams=2, bucket=200000, dim=50, loss='hs')
ft_model_Accident = fasttext.train_supervised(input="ft_ind_Accident.train", lr=0.7, epoch=25, wordNgrams=2, bucket=200000, dim=50, loss='hs')

In [52]:
ft_model_Potential.test("ft_ind_Potential.valid")

(85, 1.0, 1.0)

In [53]:
ft_model_Accident.test("ft_ind_Accident.valid")

(85, 1.0, 1.0)

In [54]:
# Juat tried with Multi labels too
ft_model_Potential = fasttext.train_supervised(input="ft_ind_Potential.train", lr=0.5, epoch=25, wordNgrams=2, bucket=200000, dim=50, loss='ova')
ft_model_Accident = fasttext.train_supervised(input="ft_ind_Accident.train", lr=0.5, epoch=25, wordNgrams=2, bucket=200000, dim=50, loss='ova')